<a href="https://colab.research.google.com/github/fernandofsilva/Challenges/blob/master/notebooks/04_modelo_lstm_sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Carregando as bibliotecas base
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style('whitegrid')

In [ ]:
#@title Carregando os dados
data = pd.read_csv(f'/content/drive/My Drive/Mestrado/data/dados_treino_teste.csv.gz', compression='gzip', index_col=0)
data.head()

# Engenharia das variáveis (Feature Engineering)

Essa sessão é composta da transformação dos dados para entrada na rede na rede neural. Portando, as variáveis são transformadas do seu valor original, seja para adequação dentro da rede neural ou para um melhor treinamento da rede, essas transformações são:

- Variavéis númericas: preco_exercicio, preco_ativo, foram normalizadas antes da entrada na rede
- Variavéis númericas: preco_opcao (alvo), volatilidade, taxa_juros e T não sofreram alterações
- Variável categórica mercado sofreu one hot encoding

A transformação dos dados é feita no mesmo momento que o modelo é treinado, isso é feito através de uma camada dentro do modelo, essa camada tem o nome de feature layer.

In [ ]:
#@title Pipeline de entrada dos dados
def df_to_dataset(dataframe, base, shuffle=True, batch_size=22):

    # Criar cópia do dataframe
    dataframe = dataframe.copy()

    # Filtrar a base
    dataframe = dataframe[dataframe['base'] == base]

    # Variavel alvo
    labels = dataframe.pop('preco_opcao')

    # Colunas do modelo
    cols = ['mercado', 'preco_exercicio', 'preco_ativo', 'T', 'volatilidade', 'taxa_juros']

    # Criar o td.data
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe[cols]), labels))

    # Embaralhar os dados se necessário
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    # Criar o batch de dados
    ds = ds.batch(batch_size)

    return ds

# Divisão da base de treino e teste
train_ds = df_to_dataset(data, base='treino')
test_ds = df_to_dataset(data, shuffle=False, base='teste')

In [ ]:
#@title Mapeamento das colunas
feature_columns = []

# Colunas númericas normalizadas
for column in ['preco_exercicio', 'preco_ativo']:
    
    mean = data.loc[data['base'] == 'treino', column].mean()
    stdev = data.loc[data['base'] == 'treino', column].std()

    feature_columns.append(tf.feature_column.numeric_column(column, normalizer_fn = lambda x: (x - mean) / stdev))

# Colunas númericas sem normalização
for column in ['T', 'volatilidade', 'taxa_juros']:

    feature_columns.append(tf.feature_column.numeric_column(column))

# Colunas categóricas
option = tf.feature_column.categorical_column_with_vocabulary_list('mercado', ['OPÇÕES DE COMPRA', 'OPÇÕES DE VENDA'])
option_one_hot = tf.feature_column.indicator_column(option)
feature_columns.append(option_one_hot)

In [ ]:
#@title Camada de transformação (feature layer)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, name='Feature')

# Modelo

O modelo de rede neural profunda a seguir, foi baseado nos estudos desenvolvidos por Hirsa, Karatas, & Oskoui. No trabalho são testadas diversas arquiteturas (camadas e elementos em cada camada), bem como função de atição de cada camada e também função de otimização.

A conclusão do estudo mostra que os melhores resultados foram obtidos utilizando uma rede de 4 camadas com 120 neurônios cada uma.

In [ ]:
#@title Criar, compilar o modelo
# Define de model
def get_model():
    model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(120, activation='elu'),
    tf.keras.layers.Dense(1)
    ])

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss='mse',
        metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE", dtype=None)]
    )

    return model

model = get_model()

In [ ]:
#@title Callbacks
# Tensorflow checkpoint
path = '/content/drive/MyDrive/Mestrado/models/dnn/'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=path+'checkpoint',
    frequecy='epoch',
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.2, min_delta=0.001, patience=5, verbose=1)

csv = tf.keras.callbacks.CSVLogger(path+"results.csv")

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=1)

In [ ]:
#@title Treinar o modelo
model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=200,
    callbacks=[checkpoint, lr, csv, es]
    )

In [ ]:
#@title Métricas
metrics = pd.read_csv(path + 'results.csv')
metrics.head()

In [ ]:
#@title Plot das Métricas do Modelo

# Valor de mudança de learning rate
metrics['lr_change'] = metrics['lr'] == metrics['lr'].shift(1)

# Valores de learning para plot de gráfico
lr_change = metrics.loc[metrics['lr_change'] == False, ['epoch', 'lr']]

# Retirar a primeira linha
lr_change = lr_change[1:]

# Converter para dict
lr_change = lr_change.to_dict('records')

# Create two subplots
fig, axs = plt.subplots(2, 1, figsize=(20, 8))
axs[0].plot(metrics['epoch'], metrics['loss'], 'tab:blue', label='treino')
axs[0].plot(metrics['epoch'], metrics['val_loss'], 'tab:red', label='teste')
axs[0].set_title('Função de Perda')
axs[0].set(ylabel='Perda')
axs[0].legend()

axs[1].plot(metrics['epoch'], metrics['MAE'], 'tab:blue', label='treino')
axs[1].plot(metrics['epoch'], metrics['val_MAE'], 'tab:red', label='teste')
axs[1].set_title('Erro Médio Absoluto')
axs[1].set(ylabel='MAE')
axs[1].legend()

for lr in lr_change:
    axs[0].axvline(x=lr['epoch'], linestyle='dotted', color='black')
    axs[1].axvline(x=lr['epoch'], linestyle='dotted', color='black')
    axs[0].text(
        x=lr['epoch'],
        y=0.1,
        s=f"lr reduzido para: {lr['lr']:.8f}",
        horizontalalignment='right'
        )
    axs[1].text(
        x=lr['epoch'],
        y=0.1,
        s=f"lr reduzido para: {lr['lr']:.8f}",
        horizontalalignment='right'
        )

for ax in axs.flat:
    ax.set(xlabel='Épocas')

for ax in axs.flat:
    ax.label_outer()

plt.show()

In [ ]:
#@title Carregar o melhor modelo
model = get_model()
model.load_weights(path+'checkpoint')

# Predição na base de teste
prediction = model.predict(test_ds)

In [ ]:
#@title Salvar predição
np.savez_compressed('/content/drive/My Drive/Mestrado/data/predicao_dnn.npz', prediction)